In [1]:
# import statements
import keras 
from keras.layers import LSTM
import pandas
import math

Using TensorFlow backend.


In [108]:
import numpy as np
from helperFunctions import isIntegerString, flattenList

''' Returns data = [[line1], [line2],...]'''
def byLine():
    data = []
    stripped_data = []
    punctuation = [",", ":", ";", ".", "?", "!", "'"]
    
    
    data = np.loadtxt("data/shakespeare.txt", delimiter='\n', dtype='bytes').astype(str)

    # Strip all whitespace (including whitespace on last line of poems)
    # Remove all punctuation 
    for i in range(len(data)):
        line = data[i].strip()
        # delete punctuation 
        for punc in punctuation:
            line = line.replace(punc, " ")
        if not isIntegerString(line):
            stripped_data.append(line)

    return stripped_data

''' Returns data = [[poem1], [poem2],...], where each poem is [[line1], [line2]...] if not flattened.
    If flattened, each poem is [line1line2....]. '''
def byPoem(flatten=False):
    data = []
    punctuation = [",", ":", ";", ".", "?", "!", "'"]
    with open('data/shakespeare.txt') as f:
        poem = []
        for line in f:
            # delete punctuation 
            for punc in punctuation:
                line = line.replace(punc, " ")
                
            if not isIntegerString(line):
                poem.append(line)
            elif len(poem) == 0: continue
            else:
                poem = poem[:-2] # Get rid of last two newlines
                poem[len(poem) - 1] = poem[len(poem) - 1].strip() # Get rid of newline at end of poem
                if flatten:
                    poem = flattenList(poem)
                data.append(poem[:])
                poem = []
    return data

''' Returns data = [[stanza1], [stanza2]...] '''
def byStanza():
    data = byPoem()
    stanzaData = []
    for poem in data:
        stanzaData.append(poem[:4])
        stanzaData.append(poem[4:8])
        stanzaData.append(poem[8:12])
        stanzaData.append(poem[12:])
    return stanzaData

In [105]:
data = byPoem()
segments = [] 
for element in data:
    current = element[0]
    n = 40
    segments.append([current[i:i+n] for i in range(0, len(current), n)])

print(segments)

[['From fairest creatures we desire increas', 'e \n'], ['When forty winters shall besiege thy bro', 'w \n'], ['Look in thy glass and tell the face thou', ' viewest \n'], ['Unthrifty loveliness why dost thou spend', ' \n'], ['Those hours that with gentle work did fr', 'ame\n'], ['Then let not winter s ragged hand deface', ' \n'], ['Lo in the orient when the gracious light', '\n'], ['Music to hear  why hear st thou music sa', 'dly \n'], ['Is it for fear to wet a widow s eye \n'], ['For shame deny that thou bear st love to', ' any\n'], ['As fast as thou shalt wane so fast thou ', 'grow st \n'], ['When I do count the clock that tells the', ' time \n'], ['O that you were your self  but love you ', 'are\n'], ['Not from the stars do I my judgement plu', 'ck \n'], ['When I consider every thing that grows\n'], ['But wherefore do not you a mightier way\n'], ['Who will believe my verse in time to com', 'e\n'], ['Shall I compare thee to a summer s day \n'], ['Devouring Time blunt thou the lion s p

In [107]:
print(len(segments))
trainingX = []
trainingY = []

print(segments[8])

for s in range(len(segments)):
    print(s)
    trainingX.append(segments[s][0])
    trainingY.append(segments[s][1])
    #trainingY.append(segments[s][1])

print(trainingX[2])
print(trainingY)

153
['Is it for fear to wet a widow s eye \n']
0
1
2
3
4
5
6
7
8


IndexError: list index out of range

In [85]:
test = ['abcdefghijklmnopqrstuvwxyz '.index(i) for i in segments[0][0].lower()]
print(segments[0][0])
print(test)

test = ['abcdefghijklmnopqrstuvwxyz '.index(i) for i in segments[1][0].lower()]
print(segments[1][0])
print(test)
test = ['abcdefghijklmnopqrstuvwxyz '.index(i) for i in segments[2][0].lower()]
print(segments[2][0])
print(test)
test = ['abcdefghijklmnopqrstuvwxyz '.index(i) for i in segments[3][0].lower()]
print(segments[3][0])
print(test)
test = ['abcdefghijklmnopqrstuvwxyz '.index(i) for i in segments[4][0].lower()]
print(segments[4][0])
print(test)
print(segments[5][0])
for i in segments[5][0].lower():
    print(i)
test2 = print(['abcdefghijklmnopqrstuvwxyz '.index(i) for i in segments[5][0].lower()])
print(test2)

From fairest creatures we desire increas
[5, 17, 14, 12, 26, 5, 0, 8, 17, 4, 18, 19, 26, 2, 17, 4, 0, 19, 20, 17, 4, 18, 26, 22, 4, 26, 3, 4, 18, 8, 17, 4, 26, 8, 13, 2, 17, 4, 0, 18]
When forty winters shall besiege thy bro
[22, 7, 4, 13, 26, 5, 14, 17, 19, 24, 26, 22, 8, 13, 19, 4, 17, 18, 26, 18, 7, 0, 11, 11, 26, 1, 4, 18, 8, 4, 6, 4, 26, 19, 7, 24, 26, 1, 17, 14]
Look in thy glass and tell the face thou
[11, 14, 14, 10, 26, 8, 13, 26, 19, 7, 24, 26, 6, 11, 0, 18, 18, 26, 0, 13, 3, 26, 19, 4, 11, 11, 26, 19, 7, 4, 26, 5, 0, 2, 4, 26, 19, 7, 14, 20]
Unthrifty loveliness why dost thou spend
[20, 13, 19, 7, 17, 8, 5, 19, 24, 26, 11, 14, 21, 4, 11, 8, 13, 4, 18, 18, 26, 22, 7, 24, 26, 3, 14, 18, 19, 26, 19, 7, 14, 20, 26, 18, 15, 4, 13, 3]
Those hours that with gentle work did fr
[19, 7, 14, 18, 4, 26, 7, 14, 20, 17, 18, 26, 19, 7, 0, 19, 26, 22, 8, 19, 7, 26, 6, 4, 13, 19, 11, 4, 26, 22, 14, 17, 10, 26, 3, 8, 3, 26, 5, 17]
Then let not winters ragged hand deface

t
h
e
n
 
l
e
t
 
n
o

ValueError: substring not found

In [39]:
# one-hot encode the labels
segments_cat = keras.utils.np_utils.to_categorical(test, num_classes = 27)

print(segments_cat[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]


In [18]:
# take sequences of 40 consecutive characters
# compare softmax output to one-hot-encoded vector representing the next character

# are we supposed to use the syllable 
model = Sequential()

# 150 LSTM units
model.add(LSTM(150, input_shape = (1, sequential.length)))

# add a dense layer
model.add(Dense(1))
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop')
model.fit(segments, epochs = 32)

NameError: name 'Sequential' is not defined

In [ ]:
# take in the first line from an actual sonnet ('seed')
# take the last 40 character from that line
# use that as your testx
# input that into your model to predct the next character
# recursively call

Hello my name is [bob]
s
Hello my name is b[obs]

hello my name is bo[bs ]
\n
hello my name is bob[s \n]